# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
filepath = os.path.join('data', 'cities.csv')

raw_data_df = pd.read_csv(filepath)
raw_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Geraldton,-28.7667,114.6000,64.81,45,0,17.27,AU,1632165675
1,Kailua,21.4022,-157.7394,84.18,73,75,11.50,US,1632165539
2,Busselton,-33.6500,115.3333,51.10,69,0,9.95,AU,1632165675
3,Rikitea,-23.1203,-134.9692,74.07,77,13,17.36,PF,1632165194
4,Acarí,-15.4311,-74.6158,70.47,61,89,9.66,PE,1632165676


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# Access maps with unique API key
gmaps.configure(api_key = g_key)

# Store data
locations = raw_data_df[['Lat', 'Lng']]
humidity = raw_data_df['Humidity']
intensity = humidity.max()

# Plot heat map
figure = gmaps.figure(center=(51.5, 0.1), zoom_level=2)

# Make a heat layer
layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=intensity, point_radius=5)

# Add the layer to the heat map
figure.add_layer(layer)

# Display
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
    * Max temperature lower than 80 degrees but higher than 70.
    * Wind speed less than 15 mph.
    * Very little cloudiness.
* Drop any rows will null values.

In [19]:
# Make a DataFrame with ideal weather conditions
critera_df = raw_data_df[(raw_data_df['Max Temp'] > 70) & (raw_data_df['Max Temp'] <= 80) & (raw_data_df['Wind Speed'] <= 15) & (raw_data_df['Cloudiness'] <= 30) & (raw_data_df['Humidity'] < 60)].dropna()
critera_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
132,Abhā,18.2164,42.5053,71.85,16,0,11.50,SA,1632165735
133,Albany,42.6001,-73.9662,76.95,47,7,7.63,US,1632165516
163,Kurtalan,37.9272,41.7028,71.82,24,0,0.76,TR,1632165749
205,Phalaborwa,-23.9430,31.1411,78.08,29,24,6.40,ZA,1632165768
212,Bethel,41.3712,-73.4140,75.31,50,20,9.22,US,1632165771
234,Southbridge,42.0751,-72.0334,74.44,43,0,5.03,US,1632165781
261,Pedras de Fogo,-7.4019,-35.1164,79.43,52,25,14.63,BR,1632165794
280,Guerrero Negro,27.9769,-114.0611,78.60,51,0,9.60,MX,1632165804
282,Notre-Dame-du-Lac,47.6136,-68.7964,70.14,41,1,7.58,CA,1632165805
472,Buin,-33.7333,-70.7500,70.12,35,0,3.36,CL,1632165895


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hotel_df = critera_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Abhā,18.2164,42.5053,71.85,16,0,11.50,SA,1632165735,
1,Albany,42.6001,-73.9662,76.95,47,7,7.63,US,1632165516,
2,Kurtalan,37.9272,41.7028,71.82,24,0,0.76,TR,1632165749,
3,Phalaborwa,-23.9430,31.1411,78.08,29,24,6.40,ZA,1632165768,
4,Bethel,41.3712,-73.4140,75.31,50,20,9.22,US,1632165771,
5,Southbridge,42.0751,-72.0334,74.44,43,0,5.03,US,1632165781,
6,Pedras de Fogo,-7.4019,-35.1164,79.43,52,25,14.63,BR,1632165794,
7,Guerrero Negro,27.9769,-114.0611,78.60,51,0,9.60,MX,1632165804,
8,Notre-Dame-du-Lac,47.6136,-68.7964,70.14,41,1,7.58,CA,1632165805,
9,Buin,-33.7333,-70.7500,70.12,35,0,3.36,CL,1632165895,


In [21]:
# Make a params dictionary to update each iteration
params = {
    'radius': 50000,
    'types': 'hotel',
    'keyword': 'hotel',
    'key': g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # change location each iteration while leaving original params in place
    params['location'] = f"{row['Lat']},{row['Lng']}"

    # Use the search term: "Hotels" and our lat/lng
    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # make request and get url
    response = requests.get(url, params=params).json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except (KeyError, IndexError):
        print('Missing field/result... skipping.')

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Abhā,18.2164,42.5053,71.85,16,0,11.50,SA,1632165735,Mercure
1,Albany,42.6001,-73.9662,76.95,47,7,7.63,US,1632165516,"The Landing Hotel, NY"
2,Kurtalan,37.9272,41.7028,71.82,24,0,0.76,TR,1632165749,REAL KONAK HOTEL
3,Phalaborwa,-23.9430,31.1411,78.08,29,24,6.40,ZA,1632165768,Unembeza Boutique Lodge & Spa
4,Bethel,41.3712,-73.4140,75.31,50,20,9.22,US,1632165771,Avon Old Farms Hotel
5,Southbridge,42.0751,-72.0334,74.44,43,0,5.03,US,1632165781,Best Western Royal Plaza Hotel & Trade Center
6,Pedras de Fogo,-7.4019,-35.1164,79.43,52,25,14.63,BR,1632165794,Hotel Manaíra
7,Guerrero Negro,27.9769,-114.0611,78.60,51,0,9.60,MX,1632165804,Hotel Los Caracoles
8,Notre-Dame-du-Lac,47.6136,-68.7964,70.14,41,1,7.58,CA,1632165805,Four Points by Sheraton Edmundston Hotel & Con...
9,Buin,-33.7333,-70.7500,70.12,35,0,3.36,CL,1632165895,ICON Hotel Santiago


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[['Lat', 'Lng']]

In [23]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
figure.add_layer(markers)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))